# Conversation History Management & Summarization System

## Project Description
This notebook demonstrates a robust system for **managing conversational chat history** between a user and an assistant. The system allows:

- **Maintaining a running conversation history** of user–assistant interactions.  
- **Summarizing conversation history** to keep it concise while preserving important context.  
- **Customizable truncation** to control the size of conversation history:
  - By **number of conversation turns** (e.g., last n messages).  
  - By **total word count** or **character length**.  
- **Periodic summarization** after every k-th turn of conversation, replacing previous messages with a concise summary for efficient memory management.  

## Initialize Groq client (OpenAI SDK compatible)

In [ ]:
from groq import Groq
client = Groq(api_key="")

## Defining Core Logic

In [2]:
class ConversationManager:
    def __init__(self, max_turns=None, max_words=None):
        self.history = []
        self.run_count = 0
        self.max_turns = max_turns
        self.max_words = max_words
    
    def add_message(self, role, content):
        """Add a message and apply truncation rules"""
        self.history.append({"role": role, "content": content})
        self.run_count += 1
        self.apply_truncation()
    
    def apply_truncation(self):
        """Apply turn-based and word-length truncation"""
        # Turn-based truncation (keep last n turns)
        if self.max_turns:
            self.history = self.history[-self.max_turns:]
        
        # Word-length truncation
        if self.max_words:
            total_words = sum(len(m['content'].split()) for m in self.history)
            while total_words > self.max_words and self.history:
                removed = self.history.pop(0)
                total_words -= len(removed['content'].split())
    
    def summarize_history(self, client, model="llama-3.1-8b-instant"):
        """Summarize conversation history"""
        if not self.history:
            return
        
        conv_text = "\n".join([f"{m['role'].capitalize()}: {m['content']}" for m in self.history])
        
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": "Summarize this conversation concisely in 2-3 sentences:"},
                    {"role": "user", "content": conv_text}
                ],
                max_tokens=100,
                temperature=0.3
            )
            summary = response.choices[0].message.content.strip()
            
            # Replace history with summary + keep last 2 exchanges for context
            last_messages = self.history[-2:] if len(self.history) >= 2 else self.history
            self.history = [{"role": "system", "content": f"Previous conversation summary: {summary}"}] + last_messages
            
        except Exception as e:
            print(f"Summarization failed: {e}")
    
    def periodic_summary(self, client, k=3, model="llama-3.1-8b-instant"):
        """Perform summarization every k-th run"""
        if self.run_count % k == 0 and self.run_count > 0:
            print(f"\n🔄 TRIGGERING PERIODIC SUMMARIZATION (Run #{self.run_count})")
            self.summarize_history(client, model=model)
    
    def get_readable_history(self):
        """Get formatted conversation history"""
        return "\n".join([f"{m['role'].capitalize()}: {m['content']}" for m in self.history])
    
    def get_stats(self):
        """Get conversation statistics"""
        total_words = sum(len(m['content'].split()) for m in self.history)
        return {
            "turns": len(self.history),
            "total_words": total_words,
            "run_count": self.run_count
        }

## DEMO 1: Basic Functionality - Multiple Conversation Samples

In [3]:
print("=" * 60)
print("DEMO 1: FEEDING MULTIPLE CONVERSATION SAMPLES")
print("=" * 60)

sample_chats = [
    ("user", "Hi, I'm Niharika from Guntur."),
    ("assistant", "Hello Niharika! Nice to meet you. How are you today?"),
    ("user", "I'm doing well, just finished my college classes."),
    ("assistant", "That's great! What are you studying?"),
    ("user", "I'm pursuing Computer Science Engineering."),
    ("assistant", "Excellent choice! Are you interested in any specific area like AI or web development?")
]

cm1 = ConversationManager(max_turns=5)

for role, msg in sample_chats:
    cm1.add_message(role, msg)

print("🔹 Conversation after feeding 6 messages (max_turns=5):")
print(cm1.get_readable_history())
print(f"\nStats: {cm1.get_stats()}")


DEMO 1: FEEDING MULTIPLE CONVERSATION SAMPLES
🔹 Conversation after feeding 6 messages (max_turns=5):
Assistant: Hello Niharika! Nice to meet you. How are you today?
User: I'm doing well, just finished my college classes.
Assistant: That's great! What are you studying?
User: I'm pursuing Computer Science Engineering.
Assistant: Excellent choice! Are you interested in any specific area like AI or web development?

Stats: {'turns': 5, 'total_words': 43, 'run_count': 6}


## DEMO 2: Word-Length Truncation

In [4]:
print("\n" + "=" * 60)
print("DEMO 2: WORD-LENGTH TRUNCATION")
print("=" * 60)

cm2 = ConversationManager(max_words=25)

for role, msg in sample_chats:
    cm2.add_message(role, msg)

print("🔹 Conversation after word-length truncation (25 words max):")
print(cm2.get_readable_history())
print(f"\nStats: {cm2.get_stats()}")


DEMO 2: WORD-LENGTH TRUNCATION
🔹 Conversation after word-length truncation (25 words max):
Assistant: That's great! What are you studying?
User: I'm pursuing Computer Science Engineering.
Assistant: Excellent choice! Are you interested in any specific area like AI or web development?

Stats: {'turns': 3, 'total_words': 25, 'run_count': 6}


## DEMO 3: Periodic Summarization (Every 3rd Run)

In [5]:
print("\n" + "=" * 60)
print("DEMO 3: PERIODIC SUMMARIZATION (Every 3rd Run)")
print("=" * 60)

cm3 = ConversationManager(max_turns=10)

short_chat = [
    ("user", "Hello!"),
    ("assistant", "Hi there! How can I help you?"),
    ("user", "I want to learn Python programming."),
    ("assistant", "Great choice! Python is beginner-friendly and versatile."),
    ("user", "Where should I start?"),
    ("assistant", "I'd recommend starting with basic syntax and data types."),
]

for i, (role, msg) in enumerate(short_chat, start=1):
    cm3.add_message(role, msg)
    cm3.periodic_summary(client, k=3)
    
    print(f"\n🔸 After Run {i}:")
    print(cm3.get_readable_history())
    print(f"Stats: {cm3.get_stats()}")



DEMO 3: PERIODIC SUMMARIZATION (Every 3rd Run)

🔸 After Run 1:
User: Hello!
Stats: {'turns': 1, 'total_words': 1, 'run_count': 1}

🔸 After Run 2:
User: Hello!
Assistant: Hi there! How can I help you?
Stats: {'turns': 2, 'total_words': 8, 'run_count': 2}

🔄 TRIGGERING PERIODIC SUMMARIZATION (Run #3)

🔸 After Run 3:
System: Previous conversation summary: There is no conversation to summarize yet. I'll start one with you. 

You've expressed interest in learning Python programming. I can provide you with resources and guidance to get started. Would you like to know where to begin or have any specific questions about Python?
Assistant: Hi there! How can I help you?
User: I want to learn Python programming.
Stats: {'turns': 3, 'total_words': 61, 'run_count': 3}

🔸 After Run 4:
System: Previous conversation summary: There is no conversation to summarize yet. I'll start one with you. 

You've expressed interest in learning Python programming. I can provide you with resources and guidance to g

## DEMO 4: COMPREHENSIVE LONG CONVERSATION (15 Turns)
# Shows all functionalities working together in a realistic scenario

In [6]:
print("\n" + "=" * 80)
print("DEMO 4: COMPREHENSIVE LONG CONVERSATION - ALL FEATURES COMBINED")
print("=" * 80)

# Create manager with combined constraints
cm_final = ConversationManager(max_turns=8, max_words=200)

# Realistic 15-turn conversation
long_conversation = [
    ("user", "Hi! I'm Niharika, a CSE student from Guntur. I need help with my final year project."),
    ("assistant", "Hello Niharika! I'd be happy to help with your final year project. What area are you focusing on?"),
    ("user", "I'm thinking about machine learning, specifically natural language processing for sentiment analysis."),
    ("assistant", "That's a great choice! Sentiment analysis is very practical. Are you planning to work with social media data like tweets or reviews?"),
    ("user", "Yes, I want to analyze movie reviews from IMDB. But I'm confused about which algorithm to use."),
    ("assistant", "For movie reviews, you could start with traditional methods like Naive Bayes or SVM, then try deep learning approaches like LSTM or BERT."),
    ("user", "I've heard about BERT but it seems complex. Is it worth the effort for a final year project?"),
    ("assistant", "BERT can be complex to implement from scratch, but using pre-trained models with libraries like Transformers makes it much easier and gives better results."),
    ("user", "That sounds good. What about the dataset size? How much data do I need for training?"),
    ("assistant", "For a good sentiment analysis model, you'd typically want at least 10,000-50,000 labeled reviews. IMDB dataset has 50k reviews which is perfect."),
    ("user", "Perfect! What programming tools should I use? I know Python basics."),
    ("assistant", "Great! You'll need Python with libraries like pandas for data handling, scikit-learn for traditional ML, and transformers/pytorch for BERT implementation."),
    ("user", "How long do you think this project will take? I have about 3 months."),
    ("assistant", "3 months is plenty! You can spend 2-3 weeks on data preprocessing, 1 month on model development and training, and remaining time on evaluation and documentation."),
    ("user", "This is really helpful! Can you suggest a timeline breakdown for better planning?"),
    ("assistant", "Sure! Week 1-2: Data collection and preprocessing. Week 3-6: Traditional ML models and BERT implementation. Week 7-10: Model comparison and optimization. Week 11-12: Documentation and presentation preparation.")
]

print("Starting comprehensive demo with 15-turn conversation...")
print("Manager settings: max_turns=8, max_words=200, periodic_summary every 4 runs\n")

for i, (role, msg) in enumerate(long_conversation, start=1):
    print(f"\n{'='*50} TURN {i} {'='*50}")
    print(f"Adding: {role.capitalize()}: {msg[:60]}{'...' if len(msg) > 60 else ''}")
    
    cm_final.add_message(role, msg)
    
    # Periodic summarization every 4 runs
    cm_final.periodic_summary(client, k=4)
    
    print(f"\n Current Conversation State:")
    print(cm_final.get_readable_history())
    
    stats = cm_final.get_stats()
    print(f"\n Stats: {stats['turns']} turns | {stats['total_words']} words | Run #{stats['run_count']}")
    
    # Show truncation effects
    if stats['turns'] == 8:
        print("  TURN LIMIT REACHED - Older messages will be truncated")
    if stats['total_words'] >= 180:
        print("  APPROACHING WORD LIMIT - May trigger word truncation")

print(f"\n{'='*80}")
print("FINAL CONVERSATION STATE AFTER ALL 15 TURNS:")
print("="*80)
print(cm_final.get_readable_history())
print(f"\nFinal Stats: {cm_final.get_stats()}")

print(f"\n{'='*80}")
print("TASK 1 DEMONSTRATION COMPLETE!")
print(" Multiple conversation samples fed")
print(" Turn-based truncation demonstrated")
print(" Word-length truncation demonstrated") 
print(" Periodic summarization every k-th run shown")
print(" All features working together in long conversation")
print("="*80)


DEMO 4: COMPREHENSIVE LONG CONVERSATION - ALL FEATURES COMBINED
Starting comprehensive demo with 15-turn conversation...
Manager settings: max_turns=8, max_words=200, periodic_summary every 4 runs


================================================== TURN 1 ==================================================
Adding: User: Hi! I'm Niharika, a CSE student from Guntur. I need help wit...

 Current Conversation State:
User: Hi! I'm Niharika, a CSE student from Guntur. I need help with my final year project.

 Stats: 1 turns | 16 words | Run #1

================================================== TURN 2 ==================================================
Adding: Assistant: Hello Niharika! I'd be happy to help with your final year pr...

 Current Conversation State:
User: Hi! I'm Niharika, a CSE student from Guntur. I need help with my final year project.
Assistant: Hello Niharika! I'd be happy to help with your final year project. What area are you focusing on?

 Stats: 2 turns | 34 words | Ru